## HW 5: Characterizing egg variation
### Author: Liz Peterson
### Date: 5-15-25


#### Step 1: Create a query that returns distinct species with egg data

In [1]:
import duckdb
import pandas as pd

conn = duckdb.connect("../database/database.db")
c = conn.cursor()

ModuleNotFoundError: No module named 'duckdb'

In [ ]:
c.execute("""SELECT DISTINCT Code, Scientific_name FROM Species 
    INNER JOIN Bird_nests
    ON Species.Code = Bird_nests.Species
    INNER JOIN Bird_eggs
    ON Bird_eggs.Nest_ID = Bird_nests.Nest_ID
    ORDER BY Scientific_name""")
c.fetchall()

#### Step 2: Iterate over the species

In [ ]:
# Define species query from above
species_query = """SELECT DISTINCT Code, Scientific_name FROM Species 
    INNER JOIN Bird_nests
    ON Species.Code = Bird_nests.Species
    INNER JOIN Bird_eggs
    ON Bird_eggs.Nest_ID = Bird_nests.Nest_ID
    ORDER BY Scientific_name"""

# Iterate over species
for row in c.execute(species_query).fetchall(): 
    species_code = row[0]
    scientific_name = row[1]

#### Step 3: Construct a query that gathers egg data for a given species, one species at a time

In [ ]:
# Define species query from above
species_query = """SELECT DISTINCT Code, Scientific_name FROM Species 
    INNER JOIN Bird_nests
    ON Species.Code = Bird_nests.Species
    INNER JOIN Bird_eggs
    ON Bird_eggs.Nest_ID = Bird_nests.Nest_ID
    ORDER BY Scientific_name"""

# Iterate over species
for row in c.execute(species_query).fetchall():
    species_code = row[0]
    scientific_name = row[1]
    egg_query = """SELECT Width*Width*Length AS Volume, Species FROM Bird_eggs
    JOIN Bird_nests
    ON Bird_eggs.Nest_ID = Bird_nests.Nest_ID
    WHERE Bird_nests.Species = ?"""
    df = pd.read_sql(egg_query, conn, params=[species_code])
    print(df)

#### Step 4: Compute statistics and print out the results

In [ ]:
# Define species query
species_query = """SELECT DISTINCT Code, Scientific_name FROM Species 
    INNER JOIN Bird_nests
    ON Species.Code = Bird_nests.Species
    INNER JOIN Bird_eggs
    ON Bird_eggs.Nest_ID = Bird_nests.Nest_ID
    ORDER BY Scientific_name"""

for row in c.execute(species_query).fetchall():
    species_code = row[0]
    scientific_name = row[1]
    egg_query = """SELECT Width*Width*Length AS Volume, Scientific_name FROM Bird_eggs
    JOIN Bird_nests
    ON Bird_eggs.Nest_ID = Bird_nests.Nest_ID
    JOIN Species
    ON Species.Code = Bird_nests.Species    
    WHERE Bird_nests.Species = ?"""
    df = pd.read_sql(egg_query, conn, params=[species_code])
    cv = round(df.Volume.std()/df.Volume.mean()*100, 2)
    print(f"{scientific_name} {cv}%")